In [ ]:
from pyspark.sql import SparkSession

In [ ]:
### example 1

In [ ]:
spark = SparkSession.builder.appName("mylogreg").getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [ ]:
my_data.show()

In [ ]:
my_log_reg_model = LogisticRegression()

In [ ]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [ ]:
log_summary = fitted_logreg.summary()

In [ ]:
log_summary.predictions.printSchema()

In [ ]:
log_summary.predictions.show()

In [ ]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [ ]:
final_model = LogisticRegression()

In [ ]:
fit_final = final_model.fit(lr_train)

In [ ]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [ ]:
prediction_and_labels.predictions.show()

In [ ]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [ ]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [ ]:
my_final_roc

In [ ]:
### example 2: titanic dataset

In [ ]:
df = spark.read.format('titanic.csv')

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
my_cols = df.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [ ]:
my_final_data = my_cols.na.drop()

In [ ]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [ ]:
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol = 'SexIndex')

In [ ]:
gender_encoder = OneHotEncoder(inputCol = 'SexIndex', outputCol = 'SexVec')

In [ ]:
embark_indexer = StringIndexer(inputCol = "Embarked", outputCol = "EmbarkIndex")
embark_encoder = OneHotEncoder(inputCol = "EmbarkIndex", outputCol = "EmbarkVec")

In [ ]:
assembler = VectorAssembler(inputCols = ['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'], outputCol = 'features')

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
log_reg_titanic = LogisticRegression(featuresCol = 'features', labelCol = 'Survived')

In [ ]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [ ]:
train_Data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [ ]:
fit_model = pipeline.fit(training_data)

In [ ]:
results = fit_model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'Survived')

In [ ]:
results.select('Survived', 'prediction').show()

In [ ]:
AUC = my_eval.evaluate(results)
AUC